In [4]:
from MySQLdb import connect
import pandas as pd

In [5]:
# Connect to your MySQLdb
conn = connect(host='localhost', user='root')

In [6]:
# Open a cursor to perform database operations
cur = conn.cursor()

The SQL SELECT TOP Clause
- The SELECT TOP clause is used to specify the number of records to return.
- The SELECT TOP clause is useful on large tables with thousands of records. Returning a large number of records can impact performance.

In [11]:
# Select only the first 3 records of the Customers table:
#cur.execute('''SELECT TOP 3 * FROM sqlw3s.Customers;''')
cur.execute('''SELECT * FROM sqlw3s.Customers LIMIT 3''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico


Note: Not all database systems support the SELECT TOP clause. MySQL supports the LIMIT clause to select a limited number of records, while Oracle uses FETCH FIRST n ROWS ONLY and ROWNUM.


>SQL Server / MS Access Syntax:
>
>SELECT TOP number|percent column_name(s)
>FROM table_name
>WHERE condition;

>MySQL Syntax:
>
>SELECT column_name(s)
>FROM table_name
>WHERE condition
>LIMIT number;

>Oracle 12 Syntax:
>
>SELECT column_name(s)
>FROM table_name
>ORDER BY column_name(s)
>FETCH FIRST number ROWS ONLY;

>Older Oracle Syntax:
>
>SELECT column_name(s)
>FROM table_name
>WHERE ROWNUM <= number;

In [15]:
# Moje - funguje i OFFSET jako u postgre
cur.execute('''SELECT * FROM sqlw3s.Customers LIMIT 3 OFFSET 2''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico
1,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
2,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


SQL TOP PERCENT Example
- The following SQL statement selects the first 50% of the records from the "Customers" table (for SQL Server/MS Access):

Example:
SELECT TOP 50 PERCENT * FROM Customers;

The following SQL statement shows the equivalent example for Oracle:
SELECT * FROM Customers
FETCH FIRST 50 PERCENT ROWS ONLY;

In [31]:
# Moje - v MySQL nejde limit percent, musi se obejit:
cur.execute('''SELECT CAST(COUNT(*) * 0.1 AS UNSIGNED) FROM sqlw3s.Customers;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,CAST(COUNT(*) * 0.1 AS UNSIGNED)
0,9


In [33]:
# Moje - v MySQL nejde limit percent, musi se obejit:
cur.execute('''SELECT FLOOR(COUNT(*) * 0.1) FROM sqlw3s.Customers;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,FLOOR(COUNT(*) * 0.1)
0,9


In [45]:
# Moje - v MySQL nejde limit percent, musi se obejit, ale neslo ani
#SELECT *
#FROM table_name.
#ORDER BY id.
#LIMIT (SELECT COUNT(*) * 0.1 FROM table_name);:
cur.execute('''SELECT * FROM sqlw3s.Customers HAVING RAND() > 0.9;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,10,Bottom-Dollar Marketse,Elizabeth Lincoln,23 Tsawassen Blvd.,Tsawassen,T2F 8M4,Canada
1,11,B's Beverages,Victoria Ashworth,Fauntleroy Circus,London,EC2 5NT,UK
2,15,Comércio Mineiro,Pedro Afonso,"Av. dos Lusíadas, 23",São Paulo,05432-043,Brazil
3,16,Consolidated Holdings,Elizabeth Brown,Berkeley Gardens 12 Brewery,London,WX1 6LT,UK
4,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany
5,33,GROSELLA-Restaurante,Manuel Pereira,5ª Ave. Los Palos Grandes,Caracas,1081,Venezuela
6,38,Island Trading,Helen Bennett,Garden House Crowther Way,Cowes,PO31 7PJ,UK
7,46,LILA-Supermercado,Carlos González,Carrera 52 con Ave. Bolívar #65-98 Llano Largo,Barquisimeto,3508,Venezuela
8,53,North/South,Simon Crowther,South House 300 Queensbridge,London,SW7 1RZ,UK
9,63,QUICK-Stop,Horst Kloss,Taucherstraße 10,Cunewalde,01307,Germany


ADD a WHERE CLAUSE
- The following SQL statement selects the first three records from the "Customers" table, where the country is "Germany" (for SQL Server/MS Access):

Example:
SELECT TOP 3 * FROM Customers
WHERE Country='Germany';

The following SQL statement shows the equivalent example for MySQL:
SELECT * FROM Customers
WHERE Country='Germany'
LIMIT 3;

The following SQL statement shows the equivalent example for Oracle:
SELECT * FROM Customers
WHERE Country='Germany'
FETCH FIRST 3 ROWS ONLY;

In [42]:
# The following SQL statement shows the equivalent example for MySQL:
cur.execute('''SELECT * FROM sqlw3s.Customers WHERE Country='Germany' LIMIT 3;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
2,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany


ADD the ORDER BY Keyword
- Add the ORDER BY keyword when you want to sort the result, and return the first 3 records of the sorted result.

For SQL Server and MS Access:

Sort the result reverse alphabetically by CustomerName, and return the first 3 records:
SELECT TOP 3 * FROM Customers
ORDER BY CustomerName DESC;


The following SQL statement shows the equivalent example for MySQL:
SELECT * FROM Customers
ORDER BY CustomerName DESC
LIMIT 3;

The following SQL statement shows the equivalent example for Oracle:
SELECT * FROM Customers
ORDER BY CustomerName DESC
FETCH FIRST 3 ROWS ONLY;

In [44]:
# The following SQL statement shows the equivalent example for MySQL:
cur.execute('''SELECT * FROM sqlw3s.Customers ORDER BY CustomerName DESC LIMIT 3;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,91,Wolski,Zbyszek,ul. Filtrowa 68,Walla,01-012,Poland
1,90,Wilman Kala,Matti Karttunen,Keskuskatu 45,Helsinki,21240,Finland
2,89,White Clover Markets,Karl Jablonski,305 - 14th Ave. S. Suite 3B,Seattle,98128,USA


In [52]:
#nejde, zkus nejak
cur.execute('''SELECT * FROM sqlw3s.Customers ORDER BY CustomerName LIMIT (SELECT FLOOR(COUNT(*) * 0.1) FROM sqlw3s.Customers) as t;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '(SELECT FLOOR(COUNT(*) * 0.1) FROM sqlw3s.Customers) as t' at line 1")